In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
from bs4 import BeautifulSoup
from openpyxl.workbook import Workbook

import os
from datetime import datetime

import requests

import pandas as pd

In [43]:
import time

In [44]:
time.sleep(1)

In [24]:
def extract_soup(url, preview=True):
    response = requests.get(url)
    status = response.status_code
    print(f'Request status: {status}')

    soup = BeautifulSoup(response.text, 'lxml')

    if preview==True:
        print(soup.prettify())

    return soup, status

In [25]:
    test_url = 'https://www.amazon.com.mx/gp/bestsellers/musical-instruments/ref=zg_bs_nav_0'
    test_soup, test_status = extract_soup(test_url, False)
    print(test_status)
    print(type(test_status))

Request status: 200
200
<class 'int'>


In [26]:
def top_amazon_boxes(soup):
    boxes = soup.find_all('div', attrs={'class':"a-section a-spacing-none aok-relative"})

    return boxes

In [45]:
def scrape_boxes(boxes, domain):
    
    ranks = [None]*50
    product_names = [None]*50
    image_urls = [None]*50
    product_links = [None]*50
    star_ratings = [None]*50
    reviews = [None]*50
    authors_companies = [None]*50
    editions_consoles = [None]*50
    min_prices = [None]*50
    max_prices = [None]*50
    time_log = [None]*50

    amz_mx_url = 'https://www.amazon.com.mx'
    
    n_box = 0
    for box in boxes:
        
        time_log[n_box] = datetime.now()

        rank_box = box.find_all('span', attrs={'class':'zg-badge-text'})
        products_and_image_box = box.find_all('div', attrs={'class' : 'a-section a-spacing-small'})
        product_links_box = box.find_all('a', attrs={'class' : 'a-link-normal'})
        star_ratings_box = box.find_all('span', attrs={'class' : 'a-icon-alt'})
        reviews_box = box.find_all('a', attrs={'class' : 'a-size-small a-link-normal'})
        authors_company_box = box.find_all('span', attrs={'class' : 'a-size-small a-color-base'})
        editions_console_box = box.find_all('span', attrs={'class' : 'a-size-small a-color-secondary'})
        prices_box = box.find_all('span', attrs={'class' : "p13n-sc-price"})
        
        
        ranks[n_box] = rank_box[0].get_text()

        #In case the element was removed (yes, it happens)
        try:
            product_names[n_box] = products_and_image_box[0].img.get('alt')
            image_urls[n_box] = products_and_image_box[0].img.get('src')
            product_links[n_box] = amz_mx_url + product_links_box[0].get('href')
        except: pass

        try:
            star_ratings[n_box] = float(star_ratings_box[0].get_text()[:3])
            reviews[n_box] = int(reviews_box[0].get_text().replace(',',''))
        except: pass
        #Individual cases
        try:
            authors_companies[n_box] = authors_company_box[0].get_text()
        except:pass
        
        try:
            editions_consoles[n_box] = editions_console_box[0].get_text()
        except:pass

        #Courrencies
        if domain == 'mx':
            coin_symbol = 1
        elif domain == 'br':
            coin_symbol = 2

        try:
            min_prices[n_box] = float(prices_box[0].get_text()[coin_symbol:].replace(',',''))
        except: pass

        try:    
            max_prices[n_box] = float(prices_box[1].get_text()[coin_symbol:].replace(',',''))
        except: pass

        n_box = n_box + 1 

    # Dictionary
    boxes_dict = {
    'time' : time_log,
    "Rank" : ranks,
    "Product Names": product_names,
    "Image urls": image_urls,
    "Product links": product_links,
    "Stars": star_ratings,
    "Reviews": reviews,
    "Authors/Company": authors_companies,
    "Edition/Console": editions_consoles,
    "Price_std_or_min" : min_prices,
    "Max_prices" : max_prices
    }

    return boxes_dict

In [28]:
mx_dict = {
    'top_food_and_drinks':'grocery',
    'top_automotive':'automotive',
    'top_baby':'baby',
    'top_sports':'sports',
    'top_amazon_devices':'amazon-devices',
    'top_electronics':'electronics',
    'top_tools':'tools',
    'top_kitchen':'kitchen',
    'top_industrial_emp_cience':'industrial',
    'top_musical_instruments':'musical-instruments',
    'top_toys':'toys',
    'top_books':'books',
    'top_music':'music',
    'top_officeproduct':'officeproduct',
    'top_movies':'dvd',
    'top_handmade':'handmade',
    'top_pet_supplies':'pet-supplies',
    'top_fashion':'shoes',
    'top_health':'hpc',
    'top_software':'software',
    'top_kindle':'digital-text',
    'top_videogames':'videogames'
    }

In [56]:
br_dict = {
    'top_amazon_devices':'amazon-devices',
    'top_baby':'baby-products',
    'top_sports':'sports',
    'top_electronics':'electronics',
    'top_tools':'hi',
    'top_kitchen':'kitchen',
    'top_toys':'toys',
    'top_books':'books',
    'top_music':'music',
    'top_officeproduct':'office',
    'top_movies':'dvd',
    'top_pet_supplies':'pet-products',
    'top_fashion':'fashion',
    'top_health':'hpc',
    'top_software':'computers',
    'top_kindle':'digital-text',
    'top_videogames':'videogames',
    'top_apps':'mobile-apps',
    'top_beauty':'beauty',
    'top_home':'home',
    'top_home_appliances':'appliances',
    'top_pool_garden':'lawn-and-garden',
    'top_furniture':'furniture'
}

In [30]:
domain_dict = {'Mexico' : 'mx',
               'Brazil' : 'br'}

In [31]:
domain_path = '4SS/4SS_db/testing/{}'

In [32]:
date = datetime.utcnow()
date

datetime.datetime(2020, 8, 1, 22, 44, 40, 37313)

In [33]:
# !ls
# %cd 'drive/My Drive/Colab Notebooks/4SS/4SS_db/testing'

In [34]:
# %cd br

In [35]:
# for categogry in br_dict:
#     %mkdir categogry

In [64]:
log_status = [None]*45
log_date = [None]*45
log_domain = [None]*45
log_category = [None]*45

n_log = 0
categories_dict = ''

for country_name in domain_dict:
    country = domain_dict[country_name]

    if country == 'mx':
        categories_dict = mx_dict

    elif country == 'br':
        categories_dict = br_dict

    for key in categories_dict:
        category = categories_dict[key]

        log_category[n_log] = category
        log_domain[n_log] = country

        url = f'https://www.amazon.com.{country}/gp/bestsellers/{category}/ref=zg_bs_nav_0'
        print(f'Scraping {category}')
        print(f'URL given: {url}')
 
        soup, status = extract_soup(url, preview=False)
        
        if status == 503:
            while status == 503:
                time.sleep(1)
                soup, status = extract_soup(url, preview=False)
                log_status[n_log] = status
                log_date[n_log] = datetime.now()

        if status ==200:
            log_status[n_log] = status
            log_date[n_log] = datetime.now()
            boxes = top_amazon_boxes(soup)

            amz_key_top_boxes = scrape_boxes(boxes, country)
            amz_key_top_boxes_df = pd.DataFrame.from_dict(amz_key_top_boxes)

            test_time = datetime.now()
            test_time = test_time.strftime('%Y-%m-%d %H_%M')

            csv_file = f'/{country}-test_{category}_{test_time}.csv'
            dir_csv_testing = f'/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing/{country}/{category}/{csv_file}'

            #Creating csv files
            amz_key_top_boxes_df.to_csv(dir_csv_testing, sep = '|', index = False)

            excel_file = f'/{country}-test_{category}_{test_time}.xlsx'
            dir_excel_testing = f'/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing/{country}/{category}/{excel_file}'
            
            #Creating xlsx files
            amz_key_top_boxes_df.to_excel(dir_excel_testing, index = False)

            print(f'Loaded {country}-{category}\n')
            time.sleep(0.05)

        else:
            print('*****************************\n')
            log_status[n_log] == status
            log_date[n_log] = datetime.now()

        n_log = n_log + 1

log_dict = {
    'log_date' : log_date,
    'category' : log_category,
    'country' : log_domain,
    'status' : log_status
}

log_df = pd.DataFrame.from_dict(log_dict)

log_file = f'/log-test_{test_time}.csv'
dir_log_testing = f'/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing/{log_file}'

log_df.to_csv(dir_log_testing, index = False)
log_df

Scraping grocery
URL given: https://www.amazon.com.mx/gp/bestsellers/grocery/ref=zg_bs_nav_0
Request status: 200
Loaded mx-grocery

Scraping automotive
URL given: https://www.amazon.com.mx/gp/bestsellers/automotive/ref=zg_bs_nav_0
Request status: 200
Loaded mx-automotive

Scraping baby
URL given: https://www.amazon.com.mx/gp/bestsellers/baby/ref=zg_bs_nav_0
Request status: 200
Loaded mx-baby

Scraping sports
URL given: https://www.amazon.com.mx/gp/bestsellers/sports/ref=zg_bs_nav_0
Request status: 200
Loaded mx-sports

Scraping amazon-devices
URL given: https://www.amazon.com.mx/gp/bestsellers/amazon-devices/ref=zg_bs_nav_0
Request status: 200
Loaded mx-amazon-devices

Scraping electronics
URL given: https://www.amazon.com.mx/gp/bestsellers/electronics/ref=zg_bs_nav_0
Request status: 200
Loaded mx-electronics

Scraping tools
URL given: https://www.amazon.com.mx/gp/bestsellers/tools/ref=zg_bs_nav_0
Request status: 200
Loaded mx-tools

Scraping kitchen
URL given: https://www.amazon.com.m

,log_date,category,country,status
0,2020-08-01 23:34:02.857836,grocery,mx,200
1,2020-08-01 23:34:03.838168,automotive,mx,200
2,2020-08-01 23:34:04.895027,baby,mx,200
3,2020-08-01 23:34:05.926839,sports,mx,200
4,2020-08-01 23:34:06.809276,amazon-devices,mx,200
5,2020-08-01 23:34:07.785254,electronics,mx,200
6,2020-08-01 23:34:09.005005,tools,mx,200
7,2020-08-01 23:34:10.165872,kitchen,mx,200
8,2020-08-01 23:34:11.105793,industrial,mx,200
9,2020-08-01 23:34:12.482069,musical-instruments,mx,200
